# Atividade de Laboratório 1
### Integrantes
- Leonardo Mallmann
- Caetano Muller
- Euzébio Hensel

## QUESTÃO 1

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.error import URLError, HTTPError, ContentTooShortError
import re
import csv
import json

: 

In [ ]:
def download(url, user_agent='wswp', charset='utf-8'):
  print('Downloading:', url)
  request = urllib.request.Request(url)
  request.add_header('User-agent', user_agent)
  try:
    resp = urllib.request.urlopen(request)
    cs = resp.headers.get_content_charset()
    if not cs:
        cs = charset
    html = resp.read().decode(cs)
  except (URLError, HTTPError, ContentTooShortError) as e:
    print('Download error:', e.reason)
  return html

In [ ]:
def scrap(html_content):
  soup = BeautifulSoup(html_content, 'html5lib')
  rows = soup.select('table tbody tr')
  tiempo = datetime.now()
  field_names = []
  field_values = []
  for row in rows:
    c = row.select('.w2p_fl label');
    field_names.append(c[0].string.strip().replace(':', ''));
    v = row.select('.w2p_fw');
    if not v[0].string:
      if len(re.findall('<img', str(v[0]))) != 0:
        img_src = v[0].select('img')[0];
        field_values.append(img_src['src'])
        continue
      if len(re.findall('<div', str(v[0]))) != 0:
        a_links = v[0].select('div a');
        neighbours = [a_tag.string.strip() for a_tag in a_links]
        field_values.append(neighbours);
        continue
      field_values.append('None')
      continue
    if v[0].string == ' ':
      field_values.append('None')
      continue
    field_values.append(v[0].string)
  field_names.append('Date/Time')
  field_values.append(tiempo)
  
  return field_values, field_names
    

### Exemplo de inserção no CSV

In [ ]:
def insert_headers(fieldnames):
  with open('places.csv', 'w', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    writer.writeheader()

def insert_data(data_row, fieldnames):
  with open('places.csv', 'a', newline='\n') as places_csv:
    writer = csv.DictWriter(places_csv, fieldnames=fieldnames)
    
    writer.writerow({
      f_name: f_value
      for f_name, f_value in zip(fieldnames,data_row)
    }) # or dict(zip(keys, values))

In [ ]:
with open('places.csv', 'r', newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    row['Capital'].replace(row['Capital'], 'Lego')
    print(row['Capital'])

In [ ]:
def crawl_sitemap(url):
  sitemap = download(url)
  links = re.findall('\d(\/.*)<\/loc>', sitemap)
  for index, link in enumerate(links):
    html = download(f'http://localhost:8000{link}')
    f_values, f_names = scrap(html)
    if index==0:
      insert_headers(f_names)
    insert_data(f_values, f_names)
    if index==4:
      break

In [ ]:
crawl_sitemap('http://localhost:8000/places/static/sitemap.xml');

In [ ]:
def page_monitor(url, csv_path):
    sitemap = download(url)
    lcsv = list(csv.reader(open(csv_path)))
    links = re.findall('\d(\/.*)<\/loc>', sitemap)
    page_number = 1;
    for index, link in enumerate(links):
        html = download(f'http://localhost:8000{link}')
        f_values, f_names = scrap(html)
        for j, value in enumerate(f_values):
            if j!=len(f_values)-1:
                if lcsv[page_number][j] != f_values[j]:
                    if isinstance(f_values[j], list):
                      if lcsv[page_number][j] != str(f_values[j]):
                        lcsv[page_number][j] = f_values[j]
                        lcsv[page_number][len(f_values)-1] = datetime.now()
                        continue
                      continue
                    lcsv[page_number][j] = f_values[j]
                    lcsv[page_number][len(f_values)-1] = datetime.now()
        if index==2:
            break
        page_number+=1
    writer = csv.writer(open(csv_path, 'w'))
    writer.writerows(lcsv)
        
    
page_monitor('http://localhost:8000/places/static/sitemap.xml', 'places.csv')

## QUESTÃO 2


In [ ]:
def cJSON(dl):
  with open('top_250_movies.json', 'a') as f:
    for dic in dl:
      json.dump(dic, f)
  


def imdb_scrap(html_content):
  soup = BeautifulSoup(html_content, 'html5lib')
  rows = soup.select('table tbody tr')
  dl = []

  for row in rows:
    #getting the posters class
    poster_a = row.select('.posterColumn a')
    #selecting image class
    img=poster_a[0].select('img')
    #getting both the redirection url and img source
    poster_url = poster_a[0]['href']
    poster_image = img[0]['src']
    #class of the title year and such...
    title_class = row.select('.titleColumn a')
    movie_title = title_class[0].string
    year_class = row.select('.titleColumn span')
    movie_year = year_class[0].string.replace('(', '').replace(')', '')
    rating_class = row.select('.ratingColumn.imdbRating strong')
    movie_rating = rating_class[0].string
    #specific_page_path = row.select('.titleColumn a')[0]['href']
    specific_page_url = f'https://www.imdb.com{poster_url}'
    specific_page_html = download(specific_page_url)
    selector = BeautifulSoup(specific_page_html, 'html5lib')
    movie_directors_tag = selector.select('.sc-fa02f843-0.fjLeDR > ul > li:first-child a')
    movie_directors_arr = [director.string for director in movie_directors_tag]
    genders_tag = selector.select('.ipc-chip-list.sc-16ede01-4.bMBIRz span')
    genders_arr = [gender.string for gender in genders_tag]
    popularity = selector.select('.sc-edc76a2-1.gopMqI')[0].string
    field_values = [movie_title, movie_year, poster_url, poster_img, movie_rating, popularity,
       movie_directors_arr, genders_arr]
    field_names = ['title', 'year', 'poster_url',
    'poster_img', 'imdb_rating', 'popularity', 'directors', 'genders']
    d = dict(zip(field_names, field_values))
    dl.append(d)
  return cJSON(dl)

  

: 